In [1]:
import torch
import numpy as np

print(f"Is GPU available? {torch.cuda.is_available()}")
print(f"PyTorch version: {torch.__version__}")

Is GPU available? True
PyTorch version: 2.8.0+cu128


### Tensors

In [2]:
tensor0d = torch.tensor(1)
tensor1d = torch.tensor([1, 2, 3])
tensor2d = torch.tensor([[1, 2],
                         [3, 4]])
tensor3d = torch.tensor([[[1, 2], [3, 4]],
                         [[5, 6], [7, 8]]])

nptensor = np.array([[1, 2], [3, 4]])
tensorfromnumpy = torch.from_numpy(nptensor)

print(tensorfromnumpy)

print("Number of tensors' dimensions:")
print(tensor0d.ndim)
print(tensor1d.ndim)
print(tensor2d.ndim)
print(tensor3d.ndim)

tensor([[1, 2],
        [3, 4]], dtype=torch.int32)
Number of tensors' dimensions:
0
1
2
3


In [3]:
floatvec = torch.tensor([1.0, 2.0, 3.0])
print("Tensors datatypes:")
print(floatvec.dtype)
print(floatvec.to(torch.int64).dtype)

Tensors datatypes:
torch.float32
torch.int64


In [4]:
print(tensorfromnumpy.shape)
floatvec.shape

torch.Size([2, 2])


torch.Size([3])

In [5]:
tensor = torch.tensor([[1, 2, 3],
                        [4, 5, 6]])
print(tensor)
tensor_reshaped = tensor.reshape(3, 2)
print(tensor_reshaped)
print(tensor.view(3, 2))

tensor([[1, 2, 3],
        [4, 5, 6]])
tensor([[1, 2],
        [3, 4],
        [5, 6]])
tensor([[1, 2],
        [3, 4],
        [5, 6]])


In [6]:
print(torch.matmul(tensor, tensor.T))
print(tensor @ tensor.T)

tensor([[14, 32],
        [32, 77]])
tensor([[14, 32],
        [32, 77]])


### Autograd

In [7]:
from torch.nn import functional as F
from torch.autograd import grad

y = torch.tensor([1.0])
x1 = torch.tensor([1.0])
w1 = torch.tensor([2.0], requires_grad=True)
b1 = torch.tensor([-1.0], requires_grad=True)

z1 = x1 * w1 + b1
a1 = torch.sigmoid(z1)
loss = F.binary_cross_entropy(a1, y)

grad_L_w1 = grad(loss, w1, retain_graph=True)
grad_L_b1 = grad(loss, b1, retain_graph=True)

print(f"Gradient of loss w.r.t w1: {grad_L_w1}")
print(f"Gradient of loss w.r.t b1: {grad_L_b1}")

Gradient of loss w.r.t w1: (tensor([-0.2689]),)
Gradient of loss w.r.t b1: (tensor([-0.2689]),)


In [8]:
loss.backward(retain_graph=True)
print(w1.grad)
print(b1.grad)

tensor([-0.2689])
tensor([-0.2689])


### Neural Network

In [9]:
class NeuralNetwork(torch.nn.Module):
    def __init__(self, num_inputs, num_outputs):
        super().__init__()

        self.layers = torch.nn.Sequential(
            torch.nn.Linear(num_inputs, 30),
            torch.nn.ReLU(),

            torch.nn.Linear(30, 20),
            torch.nn.ReLU(),

            torch.nn.Linear(20, num_outputs)
        )
    
    def forward(self, x):
        logits = self.layers(x)
        return logits

In [10]:
model = NeuralNetwork(num_inputs=50, num_outputs=3)
print(model)

NeuralNetwork(
  (layers): Sequential(
    (0): Linear(in_features=50, out_features=30, bias=True)
    (1): ReLU()
    (2): Linear(in_features=30, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=3, bias=True)
  )
)


In [11]:
num_of_train_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Number of training parameters: {num_of_train_params}")

Number of training parameters: 2213


In [12]:
print(model.layers[0].weight)
print(model.layers[0].weight.shape)

Parameter containing:
tensor([[-0.0411, -0.0360, -0.0488,  ..., -0.0826,  0.0467,  0.0860],
        [ 0.0608, -0.0954,  0.0183,  ..., -0.0947,  0.0157,  0.0394],
        [-0.0523, -0.1166,  0.1267,  ..., -0.0926, -0.0065, -0.0817],
        ...,
        [-0.0133, -0.1213,  0.0371,  ..., -0.0300,  0.1382, -0.0795],
        [ 0.0450,  0.0859,  0.1251,  ..., -0.0266,  0.1235, -0.0447],
        [ 0.1379, -0.0952, -0.0196,  ...,  0.0149,  0.0499, -0.0732]],
       requires_grad=True)
torch.Size([30, 50])


In [13]:
torch.manual_seed(123)
model = NeuralNetwork(num_inputs=50, num_outputs=3)
print(model.layers[0].weight)

Parameter containing:
tensor([[-0.0577,  0.0047, -0.0702,  ...,  0.0222,  0.1260,  0.0865],
        [ 0.0502,  0.0307,  0.0333,  ...,  0.0951,  0.1134, -0.0297],
        [ 0.1077, -0.1108,  0.0122,  ...,  0.0108, -0.1049, -0.1063],
        ...,
        [-0.0787,  0.1259,  0.0803,  ...,  0.1218,  0.1303, -0.1351],
        [ 0.1359,  0.0175, -0.0673,  ...,  0.0674,  0.0676,  0.1058],
        [ 0.0790,  0.1343, -0.0293,  ...,  0.0344, -0.0971, -0.0509]],
       requires_grad=True)


In [14]:
torch.manual_seed(123)
X = torch.randn(1, 50)
logits = model(X)
print(logits)

tensor([[-0.1080,  0.0924,  0.0775]], grad_fn=<AddmmBackward0>)


In [15]:
with torch.no_grad():
    logits = model(X)
    print(logits)

tensor([[-0.1080,  0.0924,  0.0775]])


In [16]:
with torch.no_grad():
    out = torch.softmax(model(X), dim=1)
    print(out)

tensor([[0.2919, 0.3567, 0.3514]])


### Dataloader

In [17]:
X_train = torch.tensor([
    [-1.2, 3.1],
    [-0.9, 2.9],
    [-0.5, 2.6],
    [2.3, -1.1],
    [2.7, -1.5]
])
y_train = torch.tensor([0, 0, 0, 1, 1])
X_test = torch.tensor([
    [-0.8, 2.8],
    [2.6, -1.6],
])
y_test = torch.tensor([0, 1])

In [18]:
from torch.utils.data import Dataset

class ToyDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    
    def __len__(self):
        return self.X.shape[0]
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [19]:
train_ds = ToyDataset(X_train, y_train)
test_ds = ToyDataset(X_test, y_test)

print(len(train_ds))

5


In [20]:
from torch.utils.data import DataLoader

torch.manual_seed(123)

train_loader = DataLoader(
    train_ds,
    batch_size=2,
    shuffle=True,
    num_workers=0,
    drop_last=True
)

test_loader = DataLoader(
    test_ds,
    batch_size=2,
    shuffle=False,
    num_workers=0
)

for idx, (batch_X, batch_y) in enumerate(train_loader):
    print(f"Batch {idx}:", batch_X, batch_y)

Batch 0: tensor([[ 2.3000, -1.1000],
        [-0.9000,  2.9000]]) tensor([1, 0])
Batch 1: tensor([[-1.2000,  3.1000],
        [-0.5000,  2.6000]]) tensor([0, 0])


### Training loop

In [23]:
import torch.nn.functional as F

torch.manual_seed(123)
model = NeuralNetwork(num_inputs=2, num_outputs=2)
optimizer = torch.optim.SGD(model.parameters(), lr=0.5)
num_epochs = 3

for epoch in range(num_epochs):
    model.train()
    for idx, (data, target) in enumerate(train_loader):
        output = model(data)
        loss = F.cross_entropy(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Logging
        print(f"Epoch {epoch+1:03d} / {num_epochs:03d}, Batch {idx+1:03d} / {len(train_loader):03d}, Loss: {loss.item():.2f}")

    model.eval()
print(f"Number of params: {sum(p.numel() for p in model.parameters() if p.requires_grad)} ")

Epoch 001 / 003, Batch 001 / 002, Loss: 0.75
Epoch 001 / 003, Batch 002 / 002, Loss: 0.65
Epoch 002 / 003, Batch 001 / 002, Loss: 0.44
Epoch 002 / 003, Batch 002 / 002, Loss: 0.13
Epoch 003 / 003, Batch 001 / 002, Loss: 0.03
Epoch 003 / 003, Batch 002 / 002, Loss: 0.00
Number of params: 752 


In [29]:
model.eval()
with torch.no_grad():
    output = model(X_train)
    print(output)

with torch.no_grad():
    output = model(X_train)
    probs = torch.softmax(output, dim=1)
    torch.set_printoptions(sci_mode=False)
    predictions = torch.argmax(probs, dim=1)
    print(probs)
    print(predictions)

tensor([[ 2.8569, -4.1618],
        [ 2.5382, -3.7548],
        [ 2.0944, -3.1820],
        [-1.4814,  1.4816],
        [-1.7176,  1.7342]])
tensor([[    0.9991,     0.0009],
        [    0.9982,     0.0018],
        [    0.9949,     0.0051],
        [    0.0491,     0.9509],
        [    0.0307,     0.9693]])
tensor([0, 0, 0, 1, 1])


In [32]:
def compute_accuracy(model, dataloader):
    model.eval()
    correct = 0.0
    total = 0.0

    with torch.no_grad():
        for data, target in dataloader:
            output = model(data)
            predictions = torch.argmax(output, dim=1)

            correct += (predictions == target).sum().item()
            total += len(target)
    return correct / total if total > 0 else 0.0

print(f"Train accuracy: {compute_accuracy(model, train_loader):.2f}")
print(f"Test accuracy: {compute_accuracy(model, test_loader):.2f}")


Train accuracy: 1.00
Test accuracy: 1.00


### Save model

In [33]:
torch.save(model.state_dict(), "simple_nn.pth")

In [36]:
model = NeuralNetwork(num_inputs=2, num_outputs=2)
model.load_state_dict(torch.load("simple_nn.pth"))
model

NeuralNetwork(
  (layers): Sequential(
    (0): Linear(in_features=2, out_features=30, bias=True)
    (1): ReLU()
    (2): Linear(in_features=30, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=2, bias=True)
  )
)

### Accelerate training with GPU

In [37]:
print(f"Is GPU available? {torch.cuda.is_available()}")

Is GPU available? True


In [39]:
tensor1 = torch.tensor([1.0, 2.0, 3.0])
tensor2 = torch.tensor([4.0, 5.0, 6.0])

tensor1 + tensor2

tensor([5., 7., 9.])

In [40]:
tensor1 = tensor1.to("cuda")
tensor2 = tensor2.to("cuda")

tensor1 + tensor2

tensor([5., 7., 9.], device='cuda:0')

In [ ]:
tensor1 = tensor1.to("cpu")

tensor1 + tensor2

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!